<a href="https://colab.research.google.com/github/FernandaSzAdr/Redes-Neurais--UFRPE-2019.1/blob/master/MLP/codigos/Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/FernandaSzAdr/Redes-Neurais--UFRPE-2019.1

##Multipayer Perceptron (MLP)

Rede Neural baseado no algoritmo de gradiente descendente.  
Os gradientes são calculados usando backpropagation. 

Para mais detalhes, ver os capitulos 13 a 16 do livro no site:

http://deeplearningbook.com.br/

In [0]:
!pip install ipdb
import random
import numpy as np

A entrada é uma lista (`sizes`) contém o número de neurônios nas respectivas camadas da rede. Por exemplo, se a lista for [2, 3, 1] então será uma rede de três camadas, com o primeira camada contendo 2 neurônios, a segunda camada 3 neurônios, e a terceira camada 1 neurônio. Os bias e pesos para a rede são inicializados aleatoriamente, usando uma distribuição Gaussiana com média 0 e variância 1. Note que a primeira camada é assumida como uma camada de entrada, e por convenção não definimos nenhum bias para esses neurônios, pois os bias são usados na computação das saídas das camadas posteriores.


In [0]:
def funcao_ativacao(tipo, is_prime, z):
  # Função de Ativação Sigmóide
  def sigmoid(z):
      return 1.0/(1.0+np.exp(-z))

  def relu(z):
      return np.array([[max(0, i[0])] for i in z])

  # Função para retornar as derivadas da função Sigmóide
  def sigmoid_prime(z):
      return sigmoid(z)*(1-sigmoid(z))

  def relu_prime(z):
      return relu(z)*(1-relu(z))
  
  switcher = {
      'sigmoid': sigmoid,
      'relu': relu
  }
  
  switcher_prime = {
      'sigmoid': sigmoid_prime,
      'relu': relu_prime,
  }
  
  if is_prime:
    fun = switcher_prime.get(tipo)
  else:
    fun = switcher.get(tipo)
  return fun(z)


def atualiza_taxa_aprend(taxa_inicial, epoch, n_epochs):
    return taxa_inicial * np.exp(-epoch / n_epochs)

In [0]:
# Classe Network
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        self.true_predic = {}

    def feedforward(self, a, tipo_fun):
        """Retorna a saída da rede se `a` for entrada."""
        for b, w in zip(self.biases, self.weights):
            a = funcao_ativacao(tipo_fun, False, np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None, tipo_fun='sigmoid'):
        """Treinar a rede neural usando o algoritmo mini batch com gradiente descendente. 
         A entrada é uma lista de tuplas
         `(x, y)` representando as entradas de treinamento e as
         saídas. Os outros parâmetros não opcionais são
         auto-explicativos. Se `test_data` for fornecido, então a
         rede será avaliada em relação aos dados do teste após cada
         época e progresso parcial impresso. Isso é útil para
         acompanhar o progresso, mas retarda as coisas substancialmente."""
        
        eta_inicial = eta
        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta, tipo_fun)
            
            if test_data:
                predic = self.evaluate(test_data, tipo_fun)
                self.true_predic[j] = predic
                print("Epoch {} : {} / {}".format(j,predic,n_test));
            else:
                print("Epoch {} finalizada".format(j))
            
            eta = atualiza_taxa_aprend(eta_inicial, j, epochs)
        
        return self.true_predic
    def update_mini_batch(self, mini_batch, eta, tipo_fun):
        """Atualiza os pesos e bias da rede aplicando
         a descida do gradiente usando backpropagation para um único mini lote.
         O `mini_batch` é uma lista de tuplas `(x, y)`, e `eta` é a taxa de aprendizado."""

        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y, tipo_fun)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        
        self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y, tipo_fun):
        """Retorna uma tupla `(nabla_b, nabla_w)` representando o
         gradiente para a função de custo C_x. `nabla_b` e
         `nabla_w` são listas de camadas de matrizes numpy, semelhantes
         a `self.biases` e `self.weights`."""

        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # Feedforward
        activation = x

        # Lista para armazenar todas as ativações, camada por camada
        activations = [x] 

        # Lista para armazenar todos os vetores z, camada por camada
        zs = [] 

        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = funcao_ativacao(tipo_fun, False, z)
            activations.append(activation)
        
        # Backward pass
        delta = self.cost_derivative(activations[-1], y) * funcao_ativacao(tipo_fun, True, zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        # Aqui, l = 1 significa a última camada de neurônios, l = 2 é a segunda e assim por diante. 
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = funcao_ativacao(tipo_fun, True, z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data, tipo_fun):
        """Retorna o número de entradas de teste para as quais a rede neural 
         produz o resultado correto. Note que a saída da rede neural
         é considerada o índice de qualquer que seja
         neurônio na camada final que tenha a maior ativação."""

        test_results = [(np.argmax(self.feedforward(x, tipo_fun)), np.argmax(y)) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations-y)


Como exemplo, essa mesma rede será executada na base de dados MNIST. O codigo abaixo carrega a base de dados.

In [0]:
# Carregar o dataset MNIST

# Imports
import pickle
import gzip
import numpy as np

def load_data():
    f = gzip.open('Redes-Neurais--UFRPE-2019.1/MLP/codigos/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

def all_data():
  training_data, validation_data, test_data = load_data_wrapper()

  training_data = list(training_data)
  validation_data = list(validation_data)
  test_data = list(test_data)

  return training_data + validation_data + test_data

# Mini-Projeto part. 1

1) Realizar avaliaçao usando validação cruzada

2) Modificar a taxa de aprendizagem pra diminuir com o tempo

3) Realizar avaliações com a validação cruzada modificando os seguintes parâmetros:
     
     a) Taxa de aprendizagem: 0.5, 0.1 e 0.001
     b) Função de ativação RELU
     c) 3 diferentes configurações de neurônios na camada  intermediária


Data de Entrega: 23/04/2019

In [0]:
import pandas as pd
from sklearn.model_selection import KFold

In [0]:
data = all_data()

In [0]:
media_modelos = {}
funcao_ativacao_list = []
rede = []
taxa_aprendizagem = []
medias = []

epoch = 15
n_k = 4

## Rede 1 -> Configuração 784-30-10

In [0]:
tamanho = [784, 30, 10]

### Função ativação - Relu

#### Taxa de aprendizagem - 0.5

In [21]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.5

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = sum(result.values())/epoch

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  


Epoch 0 : 1546 / 17500
Epoch 1 : 1546 / 17500
Epoch 2 : 1546 / 17500
Epoch 3 : 1546 / 17500
Epoch 4 : 1546 / 17500
Epoch 5 : 1546 / 17500
Epoch 6 : 1546 / 17500
Epoch 7 : 1546 / 17500
Epoch 8 : 1546 / 17500
Epoch 9 : 1546 / 17500
Epoch 10 : 1546 / 17500
Epoch 11 : 1546 / 17500
Epoch 12 : 1546 / 17500
Epoch 13 : 1546 / 17500
Epoch 14 : 1546 / 17500
K 1
Epoch 0 : 1604 / 17500
Epoch 1 : 1604 / 17500
Epoch 2 : 1604 / 17500
Epoch 3 : 1604 / 17500
Epoch 4 : 1604 / 17500
Epoch 5 : 1604 / 17500
Epoch 6 : 1604 / 17500
Epoch 7 : 1604 / 17500
Epoch 8 : 1604 / 17500
Epoch 9 : 1604 / 17500
Epoch 10 : 1604 / 17500
Epoch 11 : 1604 / 17500
Epoch 12 : 1604 / 17500
Epoch 13 : 1604 / 17500
Epoch 14 : 1604 / 17500
K 2
Epoch 0 : 1356 / 17500
Epoch 1 : 1356 / 17500
Epoch 2 : 1356 / 17500
Epoch 3 : 1356 / 17500
Epoch 4 : 1356 / 17500
Epoch 5 : 1356 / 17500
Epoch 6 : 1356 / 17500
Epoch 7 : 1356 / 17500
Epoch 8 : 1356 / 17500
Epoch 9 : 1356 / 17500
Epoch 10 : 1356 / 17500
Epoch 11 : 1356 / 17500
Epoch 12 : 135

In [22]:
teste1 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste1

,K,Acertos
0,0,1546.0
1,1,1604.0
2,2,1356.0
3,3,0.0


In [23]:
media = teste1['Acertos'].mean()

media

1126.5

In [0]:
funcao_ativacao_list.append('relu')
rede.append('rede 1')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.1

In [29]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.1

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = sum(result.values())/epoch

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  


Epoch 0 : 1741 / 17500
Epoch 1 : 1741 / 17500
Epoch 2 : 1741 / 17500
Epoch 3 : 1741 / 17500
Epoch 4 : 1741 / 17500
Epoch 5 : 1741 / 17500
Epoch 6 : 1741 / 17500
Epoch 7 : 1741 / 17500
Epoch 8 : 1741 / 17500
Epoch 9 : 1741 / 17500
Epoch 10 : 1741 / 17500
Epoch 11 : 1741 / 17500
Epoch 12 : 1741 / 17500
Epoch 13 : 1741 / 17500
Epoch 14 : 1741 / 17500
K 1
Epoch 0 : 1693 / 17500
Epoch 1 : 1693 / 17500
Epoch 2 : 1693 / 17500
Epoch 3 : 1693 / 17500
Epoch 4 : 1693 / 17500
Epoch 5 : 1693 / 17500
Epoch 6 : 1693 / 17500
Epoch 7 : 1693 / 17500
Epoch 8 : 1693 / 17500
Epoch 9 : 1693 / 17500
Epoch 10 : 1693 / 17500
Epoch 11 : 1693 / 17500
Epoch 12 : 1693 / 17500
Epoch 13 : 1693 / 17500
Epoch 14 : 1693 / 17500
K 2
Epoch 0 : 3998 / 17500
Epoch 1 : 3998 / 17500
Epoch 2 : 3998 / 17500
Epoch 3 : 3998 / 17500
Epoch 4 : 3998 / 17500
Epoch 5 : 3998 / 17500
Epoch 6 : 3998 / 17500
Epoch 7 : 3998 / 17500
Epoch 8 : 3998 / 17500
Epoch 9 : 3998 / 17500
Epoch 10 : 3998 / 17500
Epoch 11 : 3998 / 17500
Epoch 12 : 399

In [30]:
teste2 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste2

,K,Acertos
0,0,1741.0
1,1,1693.0
2,2,3998.0
3,3,17500.0


In [33]:
media = teste2['Acertos'].mean()

media

6233.0

In [0]:
funcao_ativacao_list.append('relu')
rede.append('rede 1')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.001

In [35]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.001

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = sum(result.values())/epoch

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: RuntimeWarning: invalid value encountered in multiply


Epoch 0 : 1741 / 17500
Epoch 1 : 1741 / 17500
Epoch 2 : 1741 / 17500
Epoch 3 : 1741 / 17500
Epoch 4 : 1741 / 17500
Epoch 5 : 1741 / 17500
Epoch 6 : 1741 / 17500
Epoch 7 : 1741 / 17500
Epoch 8 : 1741 / 17500
Epoch 9 : 1741 / 17500
Epoch 10 : 1741 / 17500
Epoch 11 : 1741 / 17500
Epoch 12 : 1741 / 17500
Epoch 13 : 1741 / 17500
Epoch 14 : 1741 / 17500
K 1
Epoch 0 : 1693 / 17500
Epoch 1 : 1693 / 17500
Epoch 2 : 1693 / 17500
Epoch 3 : 1693 / 17500
Epoch 4 : 1693 / 17500
Epoch 5 : 1693 / 17500
Epoch 6 : 1693 / 17500
Epoch 7 : 1693 / 17500
Epoch 8 : 1693 / 17500
Epoch 9 : 1693 / 17500
Epoch 10 : 1693 / 17500
Epoch 11 : 1693 / 17500
Epoch 12 : 1693 / 17500
Epoch 13 : 1693 / 17500
Epoch 14 : 1693 / 17500
K 2
Epoch 0 : 3998 / 17500
Epoch 1 : 3998 / 17500
Epoch 2 : 3998 / 17500
Epoch 3 : 3998 / 17500
Epoch 4 : 3998 / 17500
Epoch 5 : 3998 / 17500
Epoch 6 : 3998 / 17500
Epoch 7 : 3998 / 17500
Epoch 8 : 3998 / 17500
Epoch 9 : 3998 / 17500
Epoch 10 : 3998 / 17500
Epoch 11 : 3998 / 17500
Epoch 12 : 399

In [36]:
teste3 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste3

,K,Acertos
0,0,1741.0
1,1,1693.0
2,2,3998.0
3,3,17500.0


In [37]:
media = teste3['Acertos'].mean()

media

6233.0

In [0]:
funcao_ativacao_list.append('relu')
rede.append('rede 1')
taxa_aprendizagem.append(at)
medias.append(media)

### Função ativação - Sigmoid

#### Taxa de aprendizagem - 0.5

In [39]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.5

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='sigmoid')
  acertos_dict[k] = sum(result.values())/epoch

K 0
Epoch 0 : 1581 / 17500
Epoch 1 : 2563 / 17500
Epoch 2 : 2444 / 17500
Epoch 3 : 3064 / 17500
Epoch 4 : 2944 / 17500
Epoch 5 : 2775 / 17500
Epoch 6 : 2790 / 17500
Epoch 7 : 2295 / 17500
Epoch 8 : 2371 / 17500
Epoch 9 : 2483 / 17500
Epoch 10 : 2724 / 17500
Epoch 11 : 2731 / 17500
Epoch 12 : 2492 / 17500
Epoch 13 : 2459 / 17500
Epoch 14 : 3056 / 17500
K 1
Epoch 0 : 2390 / 17500
Epoch 1 : 2007 / 17500
Epoch 2 : 1895 / 17500
Epoch 3 : 1843 / 17500
Epoch 4 : 2176 / 17500
Epoch 5 : 1811 / 17500
Epoch 6 : 1962 / 17500
Epoch 7 : 1903 / 17500
Epoch 8 : 2049 / 17500
Epoch 9 : 2287 / 17500
Epoch 10 : 2218 / 17500
Epoch 11 : 2169 / 17500
Epoch 12 : 2124 / 17500
Epoch 13 : 2104 / 17500
Epoch 14 : 2378 / 17500
K 2
Epoch 0 : 3464 / 17500
Epoch 1 : 3552 / 17500
Epoch 2 : 3362 / 17500
Epoch 3 : 3517 / 17500
Epoch 4 : 3367 / 17500
Epoch 5 : 3610 / 17500
Epoch 6 : 3742 / 17500
Epoch 7 : 3579 / 17500
Epoch 8 : 3685 / 17500
Epoch 9 : 3690 / 17500
Epoch 10 : 3853 / 17500
Epoch 11 : 3683 / 17500
Epoch 12 :

In [40]:
teste4 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste4

,K,Acertos
0,0,2584.800000
1,1,2087.733333
2,2,3680.933333
3,3,2864.066667


In [41]:
media = teste4['Acertos'].mean()

media

2804.383333333333

In [0]:
funcao_ativacao_list.append('sigmoid')
rede.append('rede 1')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.1

In [43]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.1

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='sigmoid')
  acertos_dict[k] = sum(result.values())/epoch

K 0
Epoch 0 : 1709 / 17500
Epoch 1 : 2100 / 17500
Epoch 2 : 2572 / 17500
Epoch 3 : 2745 / 17500
Epoch 4 : 2884 / 17500
Epoch 5 : 2851 / 17500
Epoch 6 : 2942 / 17500
Epoch 7 : 2932 / 17500
Epoch 8 : 2959 / 17500
Epoch 9 : 3056 / 17500
Epoch 10 : 2922 / 17500
Epoch 11 : 2966 / 17500
Epoch 12 : 3040 / 17500
Epoch 13 : 3051 / 17500
Epoch 14 : 3057 / 17500
K 1
Epoch 0 : 3215 / 17500
Epoch 1 : 3190 / 17500
Epoch 2 : 3309 / 17500
Epoch 3 : 3245 / 17500
Epoch 4 : 3297 / 17500
Epoch 5 : 3316 / 17500
Epoch 6 : 3125 / 17500
Epoch 7 : 3113 / 17500
Epoch 8 : 3118 / 17500
Epoch 9 : 3016 / 17500
Epoch 10 : 3008 / 17500
Epoch 11 : 2926 / 17500
Epoch 12 : 2753 / 17500
Epoch 13 : 2778 / 17500
Epoch 14 : 2803 / 17500
K 2
Epoch 0 : 3410 / 17500
Epoch 1 : 3400 / 17500
Epoch 2 : 3298 / 17500
Epoch 3 : 3427 / 17500
Epoch 4 : 3361 / 17500
Epoch 5 : 3259 / 17500
Epoch 6 : 3167 / 17500
Epoch 7 : 3320 / 17500
Epoch 8 : 3114 / 17500
Epoch 9 : 3267 / 17500
Epoch 10 : 3254 / 17500
Epoch 11 : 3266 / 17500
Epoch 12 :

In [44]:
teste5 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste5

,K,Acertos
0,0,2785.733333
1,1,3080.800000
2,2,3280.800000
3,3,12757.200000


In [45]:
media = teste5['Acertos'].mean()

media

5476.133333333333

In [0]:
funcao_ativacao_list.append('sigmoid')
rede.append('rede 1')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.001

In [47]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.001

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='sigmoid')
  acertos_dict[k] = sum(result.values())/epoch

K 0
Epoch 0 : 2535 / 17500
Epoch 1 : 2295 / 17500
Epoch 2 : 2057 / 17500
Epoch 3 : 2035 / 17500
Epoch 4 : 2012 / 17500
Epoch 5 : 1998 / 17500
Epoch 6 : 1987 / 17500
Epoch 7 : 1981 / 17500
Epoch 8 : 1992 / 17500
Epoch 9 : 1992 / 17500
Epoch 10 : 1999 / 17500
Epoch 11 : 2003 / 17500
Epoch 12 : 2001 / 17500
Epoch 13 : 2005 / 17500
Epoch 14 : 2005 / 17500
K 1
Epoch 0 : 1885 / 17500
Epoch 1 : 1887 / 17500
Epoch 2 : 1872 / 17500
Epoch 3 : 1870 / 17500
Epoch 4 : 1873 / 17500
Epoch 5 : 1865 / 17500
Epoch 6 : 1848 / 17500
Epoch 7 : 1847 / 17500
Epoch 8 : 1844 / 17500
Epoch 9 : 1832 / 17500
Epoch 10 : 1825 / 17500
Epoch 11 : 1812 / 17500
Epoch 12 : 1799 / 17500
Epoch 13 : 1788 / 17500
Epoch 14 : 1787 / 17500
K 2
Epoch 0 : 1817 / 17500
Epoch 1 : 1790 / 17500
Epoch 2 : 1776 / 17500
Epoch 3 : 1745 / 17500
Epoch 4 : 1730 / 17500
Epoch 5 : 1718 / 17500
Epoch 6 : 1703 / 17500
Epoch 7 : 1698 / 17500
Epoch 8 : 1676 / 17500
Epoch 9 : 1666 / 17500
Epoch 10 : 1654 / 17500
Epoch 11 : 1638 / 17500
Epoch 12 :

In [48]:
teste6 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste6

,K,Acertos
0,0,2059.800000
1,1,1842.266667
2,2,1697.200000
3,3,328.600000


In [49]:
media = teste6['Acertos'].mean()

media

1481.9666666666667

In [0]:
funcao_ativacao_list.append('sigmoid')
rede.append('rede 1')
taxa_aprendizagem.append(at)
medias.append(media)

## Rede 2 -> Configuração 784-50-10

In [0]:
tamanho = [784, 50, 10]

### Função ativação - Relu

#### Taxa de aprendizagem - 0.5

In [52]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.5

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = sum(result.values())/epoch

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: RuntimeWarning: invalid value encountered in multiply


Epoch 0 : 1741 / 17500
Epoch 1 : 1741 / 17500
Epoch 2 : 1741 / 17500
Epoch 3 : 1741 / 17500
Epoch 4 : 1741 / 17500
Epoch 5 : 1741 / 17500
Epoch 6 : 1741 / 17500
Epoch 7 : 1741 / 17500
Epoch 8 : 1741 / 17500
Epoch 9 : 1741 / 17500
Epoch 10 : 1741 / 17500
Epoch 11 : 1741 / 17500
Epoch 12 : 1741 / 17500
Epoch 13 : 1741 / 17500
Epoch 14 : 1741 / 17500
K 1
Epoch 0 : 1693 / 17500
Epoch 1 : 1693 / 17500
Epoch 2 : 1693 / 17500
Epoch 3 : 1693 / 17500
Epoch 4 : 1693 / 17500
Epoch 5 : 1693 / 17500
Epoch 6 : 1693 / 17500
Epoch 7 : 1693 / 17500
Epoch 8 : 1693 / 17500
Epoch 9 : 1693 / 17500
Epoch 10 : 1693 / 17500
Epoch 11 : 1693 / 17500
Epoch 12 : 1693 / 17500
Epoch 13 : 1693 / 17500
Epoch 14 : 1693 / 17500
K 2
Epoch 0 : 3998 / 17500
Epoch 1 : 3998 / 17500
Epoch 2 : 3998 / 17500
Epoch 3 : 3998 / 17500
Epoch 4 : 3998 / 17500
Epoch 5 : 3998 / 17500
Epoch 6 : 3998 / 17500
Epoch 7 : 3998 / 17500
Epoch 8 : 3998 / 17500
Epoch 9 : 3998 / 17500
Epoch 10 : 3998 / 17500
Epoch 11 : 3998 / 17500
Epoch 12 : 399

In [53]:
teste7 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste7

,K,Acertos
0,0,1741.0
1,1,1693.0
2,2,3998.0
3,3,17500.0


In [54]:
media = teste7['Acertos'].mean()

media

6233.0

In [0]:
funcao_ativacao_list.append('relu')
rede.append('rede 2')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.1

In [56]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.1

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = sum(result.values())/epoch

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: RuntimeWarning: invalid value encountered in multiply


Epoch 0 : 1741 / 17500
Epoch 1 : 1741 / 17500
Epoch 2 : 1741 / 17500
Epoch 3 : 1741 / 17500
Epoch 4 : 1741 / 17500
Epoch 5 : 1741 / 17500
Epoch 6 : 1741 / 17500
Epoch 7 : 1741 / 17500
Epoch 8 : 1741 / 17500
Epoch 9 : 1741 / 17500
Epoch 10 : 1741 / 17500
Epoch 11 : 1741 / 17500
Epoch 12 : 1741 / 17500
Epoch 13 : 1741 / 17500
Epoch 14 : 1741 / 17500
K 1
Epoch 0 : 1693 / 17500
Epoch 1 : 1693 / 17500
Epoch 2 : 1693 / 17500
Epoch 3 : 1693 / 17500
Epoch 4 : 1693 / 17500
Epoch 5 : 1693 / 17500
Epoch 6 : 1693 / 17500
Epoch 7 : 1693 / 17500
Epoch 8 : 1693 / 17500
Epoch 9 : 1693 / 17500
Epoch 10 : 1693 / 17500
Epoch 11 : 1693 / 17500
Epoch 12 : 1693 / 17500
Epoch 13 : 1693 / 17500
Epoch 14 : 1693 / 17500
K 2
Epoch 0 : 3998 / 17500
Epoch 1 : 3998 / 17500
Epoch 2 : 3998 / 17500
Epoch 3 : 3998 / 17500
Epoch 4 : 3998 / 17500
Epoch 5 : 3998 / 17500
Epoch 6 : 3998 / 17500
Epoch 7 : 3998 / 17500
Epoch 8 : 3998 / 17500
Epoch 9 : 3998 / 17500
Epoch 10 : 3998 / 17500
Epoch 11 : 3998 / 17500
Epoch 12 : 399

In [57]:
teste8 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste8

,K,Acertos
0,0,1741.0
1,1,1693.0
2,2,3998.0
3,3,17500.0


In [58]:
media = teste8['Acertos'].mean()

media

6233.0

In [0]:
funcao_ativacao_list.append('relu')
rede.append('rede 2')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.001

In [60]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.001

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = sum(result.values())/epoch

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: RuntimeWarning: invalid value encountered in multiply


Epoch 0 : 1678 / 17500
Epoch 1 : 1678 / 17500
Epoch 2 : 1678 / 17500
Epoch 3 : 1678 / 17500
Epoch 4 : 1678 / 17500
Epoch 5 : 1678 / 17500
Epoch 6 : 1678 / 17500
Epoch 7 : 1678 / 17500
Epoch 8 : 1678 / 17500
Epoch 9 : 1678 / 17500
Epoch 10 : 1678 / 17500
Epoch 11 : 1678 / 17500
Epoch 12 : 1678 / 17500
Epoch 13 : 1678 / 17500
Epoch 14 : 1678 / 17500
K 1
Epoch 0 : 1695 / 17500
Epoch 1 : 1695 / 17500
Epoch 2 : 1695 / 17500
Epoch 3 : 1695 / 17500
Epoch 4 : 1695 / 17500
Epoch 5 : 1695 / 17500
Epoch 6 : 1695 / 17500
Epoch 7 : 1695 / 17500
Epoch 8 : 1695 / 17500
Epoch 9 : 1695 / 17500
Epoch 10 : 1695 / 17500
Epoch 11 : 1695 / 17500
Epoch 12 : 1695 / 17500
Epoch 13 : 1695 / 17500
Epoch 14 : 1695 / 17500
K 2
Epoch 0 : 1469 / 17500
Epoch 1 : 1469 / 17500
Epoch 2 : 1469 / 17500
Epoch 3 : 1469 / 17500
Epoch 4 : 1469 / 17500
Epoch 5 : 1469 / 17500
Epoch 6 : 1469 / 17500
Epoch 7 : 1469 / 17500
Epoch 8 : 1469 / 17500
Epoch 9 : 1469 / 17500
Epoch 10 : 1469 / 17500
Epoch 11 : 1469 / 17500
Epoch 12 : 146

In [61]:
teste9 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste9

,K,Acertos
0,0,1678.0
1,1,1695.0
2,2,1469.0
3,3,0.0


In [62]:
media = teste9['Acertos'].mean()

media

1210.5

In [0]:
funcao_ativacao_list.append('relu')
rede.append('rede 2')
taxa_aprendizagem.append(at)
medias.append(media)

### Função ativação - Sigmoid

#### Taxa de aprendizagem - 0.5

In [64]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.5

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='sigmoid')
  acertos_dict[k] = sum(result.values())/epoch

K 0
Epoch 0 : 1612 / 17500
Epoch 1 : 1710 / 17500
Epoch 2 : 1636 / 17500
Epoch 3 : 1671 / 17500
Epoch 4 : 1686 / 17500
Epoch 5 : 1678 / 17500
Epoch 6 : 1930 / 17500
Epoch 7 : 1615 / 17500
Epoch 8 : 1750 / 17500
Epoch 9 : 1779 / 17500
Epoch 10 : 1992 / 17500
Epoch 11 : 1907 / 17500
Epoch 12 : 1685 / 17500
Epoch 13 : 1580 / 17500
Epoch 14 : 1611 / 17500
K 1
Epoch 0 : 924 / 17500
Epoch 1 : 1769 / 17500
Epoch 2 : 2242 / 17500
Epoch 3 : 2036 / 17500
Epoch 4 : 1881 / 17500
Epoch 5 : 1416 / 17500
Epoch 6 : 1595 / 17500
Epoch 7 : 1413 / 17500
Epoch 8 : 1447 / 17500
Epoch 9 : 1104 / 17500
Epoch 10 : 1123 / 17500
Epoch 11 : 1169 / 17500
Epoch 12 : 1238 / 17500
Epoch 13 : 1354 / 17500
Epoch 14 : 1302 / 17500
K 2
Epoch 0 : 2556 / 17500
Epoch 1 : 1874 / 17500
Epoch 2 : 1914 / 17500
Epoch 3 : 2121 / 17500
Epoch 4 : 1576 / 17500
Epoch 5 : 1202 / 17500
Epoch 6 : 1687 / 17500
Epoch 7 : 1824 / 17500
Epoch 8 : 1689 / 17500
Epoch 9 : 1874 / 17500
Epoch 10 : 1861 / 17500
Epoch 11 : 1716 / 17500
Epoch 12 : 

In [65]:
teste10 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste10

,K,Acertos
0,0,1722.800000
1,1,1467.533333
2,2,1824.333333
3,3,5.866667


In [66]:
media = teste10['Acertos'].mean()

media

1255.1333333333332

In [0]:
funcao_ativacao_list.append('sigmoid')
rede.append('rede 2')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.1

In [68]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.1

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='sigmoid')
  acertos_dict[k] = sum(result.values())/epoch

K 0
Epoch 0 : 1419 / 17500
Epoch 1 : 1511 / 17500
Epoch 2 : 1598 / 17500
Epoch 3 : 1561 / 17500
Epoch 4 : 1455 / 17500
Epoch 5 : 1430 / 17500
Epoch 6 : 1446 / 17500
Epoch 7 : 1446 / 17500
Epoch 8 : 1463 / 17500
Epoch 9 : 1477 / 17500
Epoch 10 : 1470 / 17500
Epoch 11 : 1489 / 17500
Epoch 12 : 1524 / 17500
Epoch 13 : 1565 / 17500
Epoch 14 : 1588 / 17500
K 1
Epoch 0 : 1408 / 17500
Epoch 1 : 1143 / 17500
Epoch 2 : 1046 / 17500
Epoch 3 : 1099 / 17500
Epoch 4 : 1070 / 17500
Epoch 5 : 1222 / 17500
Epoch 6 : 1133 / 17500
Epoch 7 : 1144 / 17500
Epoch 8 : 1185 / 17500
Epoch 9 : 1181 / 17500
Epoch 10 : 1221 / 17500
Epoch 11 : 1267 / 17500
Epoch 12 : 1282 / 17500
Epoch 13 : 1254 / 17500
Epoch 14 : 1305 / 17500
K 2
Epoch 0 : 1035 / 17500
Epoch 1 : 884 / 17500
Epoch 2 : 1059 / 17500
Epoch 3 : 984 / 17500
Epoch 4 : 976 / 17500
Epoch 5 : 965 / 17500
Epoch 6 : 1028 / 17500
Epoch 7 : 1050 / 17500
Epoch 8 : 1077 / 17500
Epoch 9 : 1075 / 17500
Epoch 10 : 1089 / 17500
Epoch 11 : 1108 / 17500
Epoch 12 : 114

In [69]:
teste11 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste11

,K,Acertos
0,0,1496.133333
1,1,1197.333333
2,2,1049.600000
3,3,1312.666667


In [70]:
media = teste11['Acertos'].mean()

media

1263.9333333333334

In [0]:
funcao_ativacao_list.append('sigmoid')
rede.append('rede 2')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.001

In [12]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.001

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='sigmoid')
  acertos_dict[k] = sum(result.values())/epoch

K 0
Epoch 0 : 1725 / 17500
Epoch 1 : 1611 / 17500
Epoch 2 : 1580 / 17500
Epoch 3 : 1575 / 17500
Epoch 4 : 1571 / 17500
Epoch 5 : 1567 / 17500
Epoch 6 : 1554 / 17500
Epoch 7 : 1554 / 17500
Epoch 8 : 1553 / 17500
Epoch 9 : 1549 / 17500
Epoch 10 : 1545 / 17500
Epoch 11 : 1546 / 17500
Epoch 12 : 1541 / 17500
Epoch 13 : 1547 / 17500
Epoch 14 : 1546 / 17500
K 1
Epoch 0 : 1518 / 17500
Epoch 1 : 1538 / 17500
Epoch 2 : 1551 / 17500
Epoch 3 : 1562 / 17500
Epoch 4 : 1582 / 17500
Epoch 5 : 1600 / 17500
Epoch 6 : 1618 / 17500
Epoch 7 : 1620 / 17500
Epoch 8 : 1625 / 17500
Epoch 9 : 1636 / 17500
Epoch 10 : 1636 / 17500
Epoch 11 : 1634 / 17500
Epoch 12 : 1645 / 17500
Epoch 13 : 1648 / 17500
Epoch 14 : 1653 / 17500
K 2
Epoch 0 : 1673 / 17500
Epoch 1 : 1677 / 17500
Epoch 2 : 1673 / 17500
Epoch 3 : 1684 / 17500
Epoch 4 : 1693 / 17500
Epoch 5 : 1686 / 17500
Epoch 6 : 1692 / 17500
Epoch 7 : 1699 / 17500
Epoch 8 : 1702 / 17500
Epoch 9 : 1708 / 17500
Epoch 10 : 1712 / 17500
Epoch 11 : 1722 / 17500
Epoch 12 :

In [13]:
teste12 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste12

,K,Acertos
0,0,1570.933333
1,1,1604.400000
2,2,1699.933333
3,3,1817.066667


In [14]:
media = teste12['Acertos'].mean()

media

1673.0833333333333

In [0]:
funcao_ativacao_list.append('sigmoid')
rede.append('rede 2')
taxa_aprendizagem.append(at)
medias.append(media)

## Rede 3 -> Configuração 784-15-10

In [0]:
tamanho = [784, 15, 10]

### Função ativação - Relu

#### Taxa de aprendizagem - 0.5

In [17]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.5

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = sum(result.values())/epoch

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  


Epoch 0 : 1741 / 17500
Epoch 1 : 1741 / 17500
Epoch 2 : 1741 / 17500
Epoch 3 : 1741 / 17500
Epoch 4 : 1741 / 17500
Epoch 5 : 1741 / 17500
Epoch 6 : 1741 / 17500
Epoch 7 : 1741 / 17500
Epoch 8 : 1741 / 17500
Epoch 9 : 1741 / 17500
Epoch 10 : 1741 / 17500
Epoch 11 : 1741 / 17500
Epoch 12 : 1741 / 17500
Epoch 13 : 1741 / 17500
Epoch 14 : 1741 / 17500
K 1
Epoch 0 : 1693 / 17500
Epoch 1 : 1693 / 17500
Epoch 2 : 1693 / 17500
Epoch 3 : 1693 / 17500
Epoch 4 : 1693 / 17500
Epoch 5 : 1693 / 17500
Epoch 6 : 1693 / 17500
Epoch 7 : 1693 / 17500
Epoch 8 : 1693 / 17500
Epoch 9 : 1693 / 17500
Epoch 10 : 1693 / 17500
Epoch 11 : 1693 / 17500
Epoch 12 : 1693 / 17500
Epoch 13 : 1693 / 17500
Epoch 14 : 1693 / 17500
K 2
Epoch 0 : 3998 / 17500
Epoch 1 : 3998 / 17500
Epoch 2 : 3998 / 17500
Epoch 3 : 3998 / 17500
Epoch 4 : 3998 / 17500
Epoch 5 : 3998 / 17500
Epoch 6 : 3998 / 17500
Epoch 7 : 3998 / 17500
Epoch 8 : 3998 / 17500
Epoch 9 : 3998 / 17500
Epoch 10 : 3998 / 17500
Epoch 11 : 3998 / 17500
Epoch 12 : 399

In [18]:
teste7 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste7

,K,Acertos
0,0,1741.0
1,1,1693.0
2,2,3998.0
3,3,17500.0


In [19]:
media = teste7['Acertos'].mean()

media

6233.0

In [0]:
funcao_ativacao_list.append('relu')
rede.append('rede 3')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.1

In [10]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.1

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = sum(result.values())/epoch

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: RuntimeWarning: invalid value encountered in multiply


Epoch 0 : 1741 / 17500
Epoch 1 : 1741 / 17500
Epoch 2 : 1741 / 17500
Epoch 3 : 1741 / 17500
Epoch 4 : 1741 / 17500
Epoch 5 : 1741 / 17500
Epoch 6 : 1741 / 17500
Epoch 7 : 1741 / 17500
Epoch 8 : 1741 / 17500
Epoch 9 : 1741 / 17500
Epoch 10 : 1741 / 17500
Epoch 11 : 1741 / 17500
Epoch 12 : 1741 / 17500
Epoch 13 : 1741 / 17500
Epoch 14 : 1741 / 17500
K 1
Epoch 0 : 1693 / 17500
Epoch 1 : 1693 / 17500
Epoch 2 : 1693 / 17500
Epoch 3 : 1693 / 17500
Epoch 4 : 1693 / 17500
Epoch 5 : 1693 / 17500
Epoch 6 : 1693 / 17500
Epoch 7 : 1693 / 17500
Epoch 8 : 1693 / 17500
Epoch 9 : 1693 / 17500
Epoch 10 : 1693 / 17500
Epoch 11 : 1693 / 17500
Epoch 12 : 1693 / 17500
Epoch 13 : 1693 / 17500
Epoch 14 : 1693 / 17500
K 2
Epoch 0 : 3998 / 17500
Epoch 1 : 3998 / 17500
Epoch 2 : 3998 / 17500
Epoch 3 : 3998 / 17500
Epoch 4 : 3998 / 17500
Epoch 5 : 3998 / 17500
Epoch 6 : 3998 / 17500
Epoch 7 : 3998 / 17500
Epoch 8 : 3998 / 17500
Epoch 9 : 3998 / 17500
Epoch 10 : 3998 / 17500
Epoch 11 : 3998 / 17500
Epoch 12 : 399

In [11]:
teste8 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste8

,K,Acertos
0,0,1741.0
1,1,1693.0
2,2,3998.0
3,3,17500.0


In [12]:
media = teste8['Acertos'].mean()

media

6233.0

In [0]:
funcao_ativacao_list.append('relu')
rede.append('rede 3')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.001

In [14]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.001

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = sum(result.values())/epoch

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: RuntimeWarning: invalid value encountered in multiply


Epoch 0 : 1546 / 17500
Epoch 1 : 1546 / 17500
Epoch 2 : 1546 / 17500
Epoch 3 : 1546 / 17500
Epoch 4 : 1546 / 17500
Epoch 5 : 1546 / 17500
Epoch 6 : 1546 / 17500
Epoch 7 : 1546 / 17500
Epoch 8 : 1546 / 17500
Epoch 9 : 1546 / 17500
Epoch 10 : 1546 / 17500
Epoch 11 : 1546 / 17500
Epoch 12 : 1546 / 17500
Epoch 13 : 1546 / 17500
Epoch 14 : 1546 / 17500
K 1
Epoch 0 : 1604 / 17500
Epoch 1 : 1604 / 17500
Epoch 2 : 1604 / 17500
Epoch 3 : 1604 / 17500
Epoch 4 : 1604 / 17500
Epoch 5 : 1604 / 17500
Epoch 6 : 1604 / 17500
Epoch 7 : 1604 / 17500
Epoch 8 : 1604 / 17500
Epoch 9 : 1604 / 17500
Epoch 10 : 1604 / 17500
Epoch 11 : 1604 / 17500
Epoch 12 : 1604 / 17500
Epoch 13 : 1604 / 17500
Epoch 14 : 1604 / 17500
K 2
Epoch 0 : 1356 / 17500
Epoch 1 : 1356 / 17500
Epoch 2 : 1356 / 17500
Epoch 3 : 1356 / 17500
Epoch 4 : 1356 / 17500
Epoch 5 : 1356 / 17500
Epoch 6 : 1356 / 17500
Epoch 7 : 1356 / 17500
Epoch 8 : 1356 / 17500
Epoch 9 : 1356 / 17500
Epoch 10 : 1356 / 17500
Epoch 11 : 1356 / 17500
Epoch 12 : 135

In [15]:
teste9 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste9

,K,Acertos
0,0,1546.0
1,1,1604.0
2,2,1356.0
3,3,0.0


In [16]:
media = teste9['Acertos'].mean()

media

1126.5

In [0]:
funcao_ativacao_list.append('relu')
rede.append('rede 3')
taxa_aprendizagem.append(at)
medias.append(media)

### Função ativação - Sigmoid

#### Taxa de aprendizagem - 0.5

In [18]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.5

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='sigmoid')
  acertos_dict[k] = sum(result.values())/epoch

K 0
Epoch 0 : 969 / 17500
Epoch 1 : 2794 / 17500
Epoch 2 : 2059 / 17500
Epoch 3 : 2247 / 17500
Epoch 4 : 2481 / 17500
Epoch 5 : 2204 / 17500
Epoch 6 : 2404 / 17500
Epoch 7 : 2279 / 17500
Epoch 8 : 2541 / 17500
Epoch 9 : 2269 / 17500
Epoch 10 : 2420 / 17500
Epoch 11 : 2319 / 17500
Epoch 12 : 2338 / 17500
Epoch 13 : 2250 / 17500
Epoch 14 : 2312 / 17500
K 1
Epoch 0 : 2291 / 17500
Epoch 1 : 2391 / 17500
Epoch 2 : 2116 / 17500
Epoch 3 : 1885 / 17500
Epoch 4 : 1904 / 17500
Epoch 5 : 1896 / 17500
Epoch 6 : 1885 / 17500
Epoch 7 : 2229 / 17500
Epoch 8 : 2027 / 17500
Epoch 9 : 1962 / 17500
Epoch 10 : 2048 / 17500
Epoch 11 : 2251 / 17500
Epoch 12 : 2187 / 17500
Epoch 13 : 1942 / 17500
Epoch 14 : 2522 / 17500
K 2
Epoch 0 : 3378 / 17500
Epoch 1 : 3460 / 17500
Epoch 2 : 3907 / 17500
Epoch 3 : 3835 / 17500
Epoch 4 : 3442 / 17500
Epoch 5 : 3639 / 17500
Epoch 6 : 3642 / 17500
Epoch 7 : 3371 / 17500
Epoch 8 : 3646 / 17500
Epoch 9 : 3745 / 17500
Epoch 10 : 3273 / 17500
Epoch 11 : 3667 / 17500
Epoch 12 : 

In [19]:
teste10 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste10

,K,Acertos
0,0,2259.066667
1,1,2102.400000
2,2,3600.666667
3,3,17500.000000


In [20]:
media = teste10['Acertos'].mean()

media

6365.533333333333

In [0]:
funcao_ativacao_list.append('sigmoid')
rede.append('rede 3')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.1

In [22]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.1

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='sigmoid')
  acertos_dict[k] = sum(result.values())/epoch

K 0
Epoch 0 : 1968 / 17500
Epoch 1 : 2013 / 17500
Epoch 2 : 2037 / 17500
Epoch 3 : 1990 / 17500
Epoch 4 : 1734 / 17500
Epoch 5 : 1319 / 17500
Epoch 6 : 1006 / 17500
Epoch 7 : 1024 / 17500
Epoch 8 : 1107 / 17500
Epoch 9 : 1052 / 17500
Epoch 10 : 1050 / 17500
Epoch 11 : 1094 / 17500
Epoch 12 : 1298 / 17500
Epoch 13 : 2459 / 17500
Epoch 14 : 2551 / 17500
K 1
Epoch 0 : 2772 / 17500
Epoch 1 : 2972 / 17500
Epoch 2 : 3530 / 17500
Epoch 3 : 3799 / 17500
Epoch 4 : 3975 / 17500
Epoch 5 : 4059 / 17500
Epoch 6 : 4042 / 17500
Epoch 7 : 4039 / 17500
Epoch 8 : 4026 / 17500
Epoch 9 : 4006 / 17500
Epoch 10 : 3971 / 17500
Epoch 11 : 4003 / 17500
Epoch 12 : 3954 / 17500
Epoch 13 : 3873 / 17500
Epoch 14 : 3832 / 17500
K 2
Epoch 0 : 3279 / 17500
Epoch 1 : 3129 / 17500
Epoch 2 : 3146 / 17500
Epoch 3 : 3335 / 17500
Epoch 4 : 3296 / 17500
Epoch 5 : 3390 / 17500
Epoch 6 : 3526 / 17500
Epoch 7 : 3445 / 17500
Epoch 8 : 3644 / 17500
Epoch 9 : 3599 / 17500
Epoch 10 : 3730 / 17500
Epoch 11 : 3891 / 17500
Epoch 12 :

In [23]:
teste11 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste11

,K,Acertos
0,0,1580.133333
1,1,3790.200000
2,2,3555.333333
3,3,1984.933333


In [24]:
media = teste11['Acertos'].mean()

media

2727.6499999999996

In [0]:
funcao_ativacao_list.append('sigmoid')
rede.append('rede 3')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.001

In [26]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.001

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='sigmoid')
  acertos_dict[k] = sum(result.values())/epoch

K 0
Epoch 0 : 1610 / 17500
Epoch 1 : 1544 / 17500
Epoch 2 : 1511 / 17500
Epoch 3 : 1509 / 17500
Epoch 4 : 1493 / 17500
Epoch 5 : 1487 / 17500
Epoch 6 : 1480 / 17500
Epoch 7 : 1469 / 17500
Epoch 8 : 1450 / 17500
Epoch 9 : 1443 / 17500
Epoch 10 : 1428 / 17500
Epoch 11 : 1426 / 17500
Epoch 12 : 1418 / 17500
Epoch 13 : 1418 / 17500
Epoch 14 : 1412 / 17500
K 1
Epoch 0 : 1385 / 17500
Epoch 1 : 1373 / 17500
Epoch 2 : 1358 / 17500
Epoch 3 : 1348 / 17500
Epoch 4 : 1350 / 17500
Epoch 5 : 1333 / 17500
Epoch 6 : 1341 / 17500
Epoch 7 : 1332 / 17500
Epoch 8 : 1327 / 17500
Epoch 9 : 1320 / 17500
Epoch 10 : 1299 / 17500
Epoch 11 : 1291 / 17500
Epoch 12 : 1288 / 17500
Epoch 13 : 1287 / 17500
Epoch 14 : 1276 / 17500
K 2
Epoch 0 : 1683 / 17500
Epoch 1 : 1665 / 17500
Epoch 2 : 1665 / 17500
Epoch 3 : 1659 / 17500
Epoch 4 : 1652 / 17500
Epoch 5 : 1642 / 17500
Epoch 6 : 1650 / 17500
Epoch 7 : 1642 / 17500
Epoch 8 : 1631 / 17500
Epoch 9 : 1627 / 17500
Epoch 10 : 1620 / 17500
Epoch 11 : 1630 / 17500
Epoch 12 :

In [27]:
teste12 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste12

,K,Acertos
0,0,1473.200000
1,1,1327.200000
2,2,1643.066667
3,3,5273.800000


In [28]:
media = teste12['Acertos'].mean()

media

2429.3166666666666

In [0]:
funcao_ativacao_list.append('sigmoid')
rede.append('rede 3')
taxa_aprendizagem.append(at)
medias.append(media)

## Resultados 1 Entrega

In [30]:
resultado_total = pd.DataFrame({
    'rede': rede,
    'funcao de ativacao': funcao_ativacao_list,
    'taxa de aprendizagem': taxa_aprendizagem,
    'media k-fold': medias
})

resultado_total

,rede,funcao de ativacao,taxa de aprendizagem,media k-fold
0,rede 3,relu,0.100,6233.000000
1,rede 3,relu,0.001,1126.500000
2,rede 3,sigmoid,0.500,6365.533333
3,rede 3,sigmoid,0.100,2727.650000
4,rede 3,sigmoid,0.001,2429.316667


# Mini-Projeto part. 2

A partir dos melhores resultados obtidos na etapa anterior, resolver as seguintes questões:

4) Implementar as regularizações L1 e L2

5) Implementar o Momento

6) Comparar os experimentos e explicar o que achou dos resultados.

Data de Entrega: 30/04/2019